# OpenAI Function Calling


**Notes**:
- LLMs don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.
- OpenAI has announced the release of an updated GPT-3.5-Turbo model and the deprecation of the ```gpt-3.5-turbo-0613``` model. The gpt-3.5-turbo-0613 model, which has been utilized in this Short Course since its launch in October 2023, will be replaced by the ```gpt-3.5-turbo``` model.

In [235]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [236]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)
def extrapolate(initial = {"initial":"0"}):
    """Extrapolate the number"""
    print(initial)
    extrapolated = {
        "initial" : initial["initial"],
        "next_number": str(float(initial["initial"]) + 1),
    }
    return json.dumps(extrapolated)
    

In [237]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    },
    {
        "name" : "extrapolate",
        "description": "Extrapolate the number",
        "parameters": {
            "type" : "object",
            "properties" : {
                "initial":{"type" : "string"}
            }
            
        }
    }
]

In [238]:
messages = [
    {
        "role": "user",
        "content": "Please extrapolate 22.3?"
    }
]

In [239]:
import openai

> Note: This notebook was updated in June 2024. Consequently, we are now using the ```gpt-3.5-turbo model``` instead of the ```gpt-3.5-turbo-0613``` model featured by the instructor in the video lesson.

In [240]:
# Call the ChatCompletion endpoint
response = openai.ChatCompletion.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

> Note: The following result may differ slightly from the one shown by the instructor in the video lesson due to the model being updated.

In [241]:
print(response)

{
  "id": "chatcmpl-C61cLxtGLxLu4Q3xdqlGEylhJPdIy",
  "object": "chat.completion",
  "created": 1755552221,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extrapolate",
          "arguments": "{\"initial\":\"22.3\"}"
        },
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 103,
    "completion_tokens": 18,
    "total_tokens": 121,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


In [242]:
response_message = response["choices"][0]["message"]

In [243]:
response_message

<OpenAIObject at 0x7f9bb71743b0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "extrapolate",
    "arguments": "{\"initial\":\"22.3\"}"
  },
  "refusal": null,
  "annotations": []
}

In [244]:
response_message["content"]

In [245]:
response_message["function_call"]

<OpenAIObject at 0x7f9b8ed33040> JSON: {
  "name": "extrapolate",
  "arguments": "{\"initial\":\"22.3\"}"
}

In [246]:
json.loads(response_message["function_call"]["arguments"])

{'initial': '22.3'}

In [247]:
args = json.loads(response_message["function_call"]["arguments"])

In [248]:
get_current_weather(args)

'{"location": {"initial": "22.3"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

* Pass a message that is not related to a function.

In [249]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

> Note: This notebook was updated in June 2024. Consequently, we are now using the ```gpt-3.5-turbo model``` instead of the ```gpt-3.5-turbo-0613``` model featured by the instructor in the video lesson.

In [250]:
response = openai.ChatCompletion.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
)

In [251]:
print(response)

{
  "id": "chatcmpl-C61d8lrJm82yrgNipVqZ61GhdgzCX",
  "object": "chat.completion",
  "created": 1755552270,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 97,
    "completion_tokens": 10,
    "total_tokens": 107,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


* Pass additional parameters to force the model to use or not a function.

In [252]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-C61dEGoZpO2jSwzNfm058gNuw8xjf",
  "object": "chat.completion",
  "created": 1755552276,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 97,
    "completion_tokens": 10,
    "total_tokens": 107,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


* Use mode 'none' for function call.

In [253]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-C61dGVrbzryGViHpKLBiWd8ock2pY",
  "object": "chat.completion",
  "created": 1755552278,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 98,
    "completion_tokens": 9,
    "total_tokens": 107,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


* When the message should call a function and still uses mode 'none'.

In [254]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-C61dZbos5s6YsO7oaFyIYCDYST4HB",
  "object": "chat.completion",
  "created": 1755552297,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Please give me a moment to fetch the current weather in Boston.",
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 103,
    "completion_tokens": 13,
    "total_tokens": 116,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


* Force calling a function.

In [255]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call={"name": "extrapolate"},
)
print(response)

{
  "id": "chatcmpl-C61dwN0TEe8Dt0YcrMXYA2tjJKpo1",
  "object": "chat.completion",
  "created": 1755552320,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extrapolate",
          "arguments": "{\"initial\":\"Hello\"}"
        },
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 108,
    "completion_tokens": 5,
    "total_tokens": 113,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


* Final notes.

In [256]:
messages = [
    {
        "role": "user",
        "content": "Please extrapolate 22.3",
    }
]
response = openai.ChatCompletion.create(
    # OpenAI Updates: As of June 2024, we are now using the GPT-3.5-Turbo model
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call={"name": "extrapolate"},
)
print(response)

{
  "id": "chatcmpl-C61i2JLodMar0bvsF4fV8aAVEcPox",
  "object": "chat.completion",
  "created": 1755552574,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "extrapolate",
          "arguments": "{\"initial\":\"22.3\"}"
        },
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 113,
    "completion_tokens": 7,
    "total_tokens": 120,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}


In [257]:
messages.append(response["choices"][0]["message"])

In [258]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = extrapolate(args)

{'initial': '22.3'}


In [259]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [260]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
)
print(response)

{
  "id": "chatcmpl-C61tVdHxyqxUkoNmMZJK7RxDmrbIG",
  "object": "chat.completion",
  "created": 1755553285,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Extrapolating from 22.3, the next number in the sequence would be 23.3.",
        "refusal": null,
        "annotations": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 58,
    "completion_tokens": 23,
    "total_tokens": 81,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "service_tier": "default",
  "system_fingerprint": null
}
